<a href="https://colab.research.google.com/github/jhmuller/mlp-covid19/blob/main/mlp_covid19_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Notes

#### enable shift-tab
On Colab, for SHIFT+ TAB to work, you have to disable Automatically trigger code completions on Tools->Settings

You also have to have already imported the library that you are trying to get the docstring. 

For example run the import in another code snippet. You can see the docstring also just pressing TAB on Colab

---



In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install fsspec

     |████████████████████████████████| 92kB 6.6MB/s 


In [44]:
import os
import sys
import datetime
import unicodedata
import string
import re
import random
import json
import glob
import fsspec
from __future__ import unicode_literals, print_function, division
import re
import nltk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
now = datetime.datetime.now
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import inspect
mlist = list(filter(lambda x: inspect.ismodule(x[1]), locals().items()))
vi = sys.version_info
print("version {0}.{1}.{2} of Python".format(vi.major, vi.minor, vi.micro))
for name, mod in mlist:
    mname = name
    if name.startswith("__"):
        print(name, mod)
        continue
    if hasattr(mod, "__version__"):
        mname = name
        if hasattr(mod, "__path__"):
            mname = os.path.split(mod.__path__[0])[1]
        print("version {1} of {0} as {2} ".format(mname, name, mod.__version__))
    elif hasattr(mod, "__file__") and "site-packages" in mod.__file__:
        print("No __version__ for {0} as {1}".format(mname, name))
del mod
del name

version 3.6.9 of Python
__builtin__ <module 'builtins' (built-in)>
__builtins__ <module 'builtins' (built-in)>
version re of re as 2.2.1 
version json of json as 2.0.9 
version fsspec of fsspec as 0.8.4 
version nltk of nltk as 3.2.5 
version pd of pandas as 1.1.4 
version np of numpy as 1.18.5 
version torch of torch as 1.7.0+cu101 


In [6]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [14]:
#%%timeit -n1 -r1
# tried using json and pandas but had problems
# just reading in text and splitting into lines
dpath = "/content/gdrive/My Drive/data/mlp_covid19"
fdir = os.listdir(dpath)[0]
dpath = os.path.join(dpath, fdir)
jfiles = os.listdir(dpath)
jpaths = [os.path.join(dpath, jf) for jf in jfiles]

# this did not work to read as json
if False:
  with open(jpaths[0]) as f:
    data = json.load(f)

#To make it easier to work on your computer, we will work with only the 1st 2 files. 
# i.e from 16119_webhose_2019_12_db21c91a1ab47385bb13773ed8238c31_0000001.json 
# and 16119_webhose_2020_01_db21c91a1ab47385bb13773ed8238c31_0000001.json.
files_set = set(
    ["16119_webhose_2019_12_db21c91a1ab47385bb13773ed8238c31_0000001.json",
      "16119_webhose_2020_01_db21c91a1ab47385bb13773ed8238c31_0000001.json"])
txt_data = ''
for fpath in jpaths:
  head, tail = os.path.split(fpath)
  if tail in files_set:
    print("reading {0} {1}".format(tail, datetime.datetime.now()))
    with open(jpaths[0], mode='r') as fp:
      new_data = fp.read()
    print("len {0} {1}".format(len(new_data), datetime.datetime.now()))
    txt_data += new_data

print("len of txt data {0}".format(len(txt_data)))
print(datetime.datetime.now())
tvars = [x for x in globals().keys() if  x.startswith("t")]
print(tvars)

reading 16119_webhose_2019_12_db21c91a1ab47385bb13773ed8238c31_0000001.json 2020-12-05 18:17:32.257237
len 521195604 2020-12-05 18:17:35.720087
reading 16119_webhose_2020_01_db21c91a1ab47385bb13773ed8238c31_0000001.json 2020-12-05 18:17:35.720333
len 521195604 2020-12-05 18:17:39.177788
len of txt data 1042391208
2020-12-05 18:17:40.459928
['torch', 'tail', 'txt_data']


In [18]:
txt_lines = txt_data.split('\n')
print("{0} lines".format(len(txt_lines)))
dlist = []
for i, l in enumerate(txt_lines):
  try:
    d = eval(l) 
    dlist.append(d)
  except Exception as e:
    print("line{0}, {1}\n{2}".format(i,l, e))
print("len dlist {0}".format(len(dlist)))

197913 lines
line98956, {"organizations": [], "uuid": "50db3ae27600471c975e2b3f9ffae3693f861967", "thread": {"social": {"gplus": {"shares": 0}, "pinterest": {"shares": 0}, "vk": {"shares": 0}, "linkedin": {"shares": 0}, "facebook": {"likes": 0, "shares": 0, "comments": 0}, "stumbledupon": {"shares": 0}}, "site_full": "www.businessinsider.sg", "main_image": "https://static.businessinsider.sg/sites/2/2020/03/5c24ee842a5b7429c428e9cc.png", "site_section": "https://www.businessinsider.sg/career", "section_title": "Latest Career on Business Insider Singapore - Business Insider Singapore", "url": "https://www.businessinsider.sg/amazon-walmart-hiring-how-to-apply-2020-3", "country": "SG", "domain_rank": 26759, "title": "Amazon and Walmart are ramping up hiring to add 250,000 new jobs. Here's how to apply and whether you can expect an interview.,", "performance_score": 0, "site": "businessinsider.sg", "participants_count": 1, "title_full": "", "spam_score": 0.027, "site_type": "news", "publish

In [19]:
#%%timeit -n1 -r1
dataset = [d['text'] for d in dlist]
target = [d['title'] for d in dlist]
# The length of the list dataset and target will be 190138.
print("dataset len= {0}, target len= {1}".format(len(dataset), len(target)))

dataset len= 197912, target len= 197912


In [20]:
print(datetime.datetime.now())
dataset[0:5]
target[0:5]

2020-12-05 18:34:45.803556


['COVID-19: Goa SEC cancels March 24 ZP polls; board exams postponed',
 'EarthLink - News',
 'Administrative Leave Offered to County Workers',
 '',
 'Unanimous first-teamer Obi Toppin leads 2019-20 college basketball AP All-American team']

In [21]:
contraction_map = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [25]:
# this did not work
# from nltk.corpus import stopwords
# stop_words = set(nltk.corpus.stopwords.words('english'))
# so doing this
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))  
print(type(stop_words))
print(list(stop_words)[:4])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<class 'set'>
['about', 'then', 'more', 'hasn']


In [26]:
def preprocess(text, contraction_map= None, stop_words = None):
  if not contraction_map:
    contraction_map = {}
  if not stop_words:
    stop_words = []
  if not isinstance(stop_words, set):
    stop_words = set(stop_words)

  # Split the text using Python split() function
  toks = text.split()
  # Apply the contraction hashmap on all the words of the text
  toks = [contraction_map[x] if x in contraction_map.keys() else x for x in toks]  
  # Remove the stopwords that are in English
  toks = [x for x in toks if x not in stop_words]

  # now rejoin to apply other transforms on text string
  res = ' '.join(toks)
  # Convert text to lowercase
  res = res.lower()    
  # Remove 's. For example yours becomes your
  res = res.replace("'s",'') # convert your's -> your
  # Use regular expression to remove parentheses outside a word. For example (word) becomes word
  res = re.sub("\(|\)", "", res) 
  # Use regular expression to remove punctuations
  res = re.sub(r'[^a-zA-Z0-9. ]','',res)
  # Use regular expression to add a space character before and after the full stop. For example . becomes .
  res = re.sub("\.", " . ", res)
  return res
test = "The boys can't .(go) there!."
test_out = preprocess(test, 
           contraction_map=contraction_map, 
           stop_words=stop_words)
print("{0}\n{1}".format(test, test_out))

The boys can't .(go) there!.
the boys cannot  . go there . 


In [27]:
def preprocess_ref(text, contraction_map, stop_words):
    text = text.lower() # lowercase
    text = text.split() # convert have'nt -> have not
    for i in range(len(text)):
        word = text[i]
        if word in contraction_map:
            text[i] = contraction_map[word]
    text = " ".join(text)
    text = text.split()
    newtext = []
    for word in text:
        if word not in stop_words:
            newtext.append(word)
    text = " ".join(newtext)
    text = text.replace("'s",'') # convert your's -> your
    text = re.sub(r'\(.*\)','',text) # remove (words)
    text = re.sub(r'[^a-zA-Z0-9. ]','',text) # remove punctuations
    text = re.sub(r'\.',' . ',text)
    return text
test = "The boys can't .(go) there!."
test_out = preprocess(test, 
           contraction_map=contraction_map, 
           stop_words=stop_words)
print("{0}\n{1}".format(test, test_out))    

The boys can't .(go) there!.
the boys cannot  . go there . 


In [50]:
%time X = [preprocess(line, contraction_map=contraction_map, stop_words=stop_words) for line in dataset]

CPU times: user 47.9 s, sys: 827 ms, total: 48.7 s
Wall time: 48.7 s


In [51]:
%time X_ref = [preprocess_ref(line, contraction_map=contraction_map, stop_words=stop_words) for line in dataset]

CPU times: user 48.6 s, sys: 751 ms, total: 49.3 s
Wall time: 49.3 s


In [52]:
%time Y = [preprocess(line, contraction_map=contraction_map, stop_words=stop_words) for line in target]

CPU times: user 1.7 s, sys: 32.8 ms, total: 1.73 s
Wall time: 1.73 s


In [53]:
%time Y_ref = [preprocess_ref(line, contraction_map=contraction_map, stop_words=stop_words) for line in target]

CPU times: user 1.75 s, sys: 34.8 ms, total: 1.78 s
Wall time: 1.78 s


In [54]:
# At this point, the length of X and Y should both be 190138.
print("len x= {0}, len(y)= {1}".format(len(x), len(y)))
print("len x_ref= {0}, len y_ref= {1}".format(len(x_ref), len(y_ref)))
#assert(len(x) == 190138)

len x= 197912, len(y)= 197912
len x_ref= 197912, len y_ref= 197912


In [55]:
short_text=[]
short_summary=[]
max_len_text = 600
max_len_target = 30
for i in range(len(dataset)):
    if(len(target[i].split())<=max_len_target and len(dataset[i].split())<=max_len_text):
        short_text.append(dataset[i])
        short_summary.append(target[i])

temp_df=pd.DataFrame({'text':short_text,'summary':short_summary})
print("temp_df shape {0}".format(temp_df.shape))
# len(temp_df) being 130736.

temp_df shape (143468, 2)


In [56]:
temp_df.head(2)

,text,summary
0,India News COVID-19: Goa SEC cancels March 24 ...,COVID-19: Goa SEC cancels March 24 ZP polls; b...
1,John Legend: 'The Voice' episodes taped until ...,EarthLink - News


In [57]:
# remove empty strings from summary and the text column.
newdf = temp_df[temp_df['summary'].str.strip().astype(bool)]
df = newdf[newdf['text'].str.strip().astype(bool)]
print("df shape {0}".format(df.shape))
print(df.head(3))

df shape (130950, 2)
                                                text                                            summary
0  India News COVID-19: Goa SEC cancels March 24 ...  COVID-19: Goa SEC cancels March 24 ZP polls; b...
1  John Legend: 'The Voice' episodes taped until ...                                   EarthLink - News
2  Administrative Leave Offered to County Workers...     Administrative Leave Offered to County Workers


In [58]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 20

SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [59]:
df.head(2)

,text,summary
0,India News COVID-19: Goa SEC cancels March 24 ...,COVID-19: Goa SEC cancels March 24 ZP polls; b...
1,John Legend: 'The Voice' episodes taped until ...,EarthLink - News


In [64]:

def readData(text, summary):
  print("Reading lines...")
  input_lang = Lang(text)
  output_lang = Lang(summary)
  pairs = [[text[i],summary[i]] for i in range(len(text))]
  return (input_lang, output_lang, pairs)

def prepareData(tlist, slist):
  input_lang, output_lang, pairs = readData(tlist, slist)
  print("Read %s sentence pairs" % len(pairs))
  print("Counting words...")  
  [input_lang.addSentence(t) for t in tlist]
  [output_lang.addSentence(s) for s in slist] 
  return input_lang, output_lang, pairs 

In [66]:
input, output, pairs = prepareData(X,Y)

Reading lines...
Read 197912 sentence pairs
Counting words...
